In [3]:
f = open('conll2017/all/task1/finnish-train-low').read().splitlines()

In [4]:
from collections import defaultdict

fk = defaultdict(dict)
inti = 0
for item in f:
    stuff = item.split('\t')
    fk[inti]['source'] = stuff[0]
    fk[inti]['inf'] = stuff[1]
    stuff2 = stuff[2].split(';')
    fk[inti]['pos'] = stuff2[0]
    fk[inti]['num'] = stuff2[-1]
    fk[inti]['case'] = '-'.join(stuff2[1:-2])
    inti += 1

In [6]:
import pandas
pd = pandas.DataFrame.from_dict(fk,orient='index')
# Ignore the statements below
nouns = pd[pd['pos']=='N'].dropna(axis=1)
nouns.groupby(['num','case']).count()
pd[pd['pos']=='N'].dropna(axis=1)
nouns[(nouns['num']=='PL')].head(5)
data = pandas.read_csv('data/task1/output/source_csvs/finnish-train-low')
# del data['prop']
del data['Unnamed: 0']
pd = data[data['pos'] == 'N'].dropna(axis=1)

In [10]:
from IPython.display import display

rw3= pandas.DataFrame()
transM = pandas.DataFrame()
for i,item in pd.iterrows():
    charPairs = list()
    sourLen = len(item['source'])
    for j,chara in enumerate(item['source']):
        charPairs.append([i,chara,j+1,(sourLen - j)*-1])
        
    tempDF = pandas.DataFrame.from_records(charPairs,columns=['indi','chara','lpos','rpos'])
    tempTrans = pandas.merge(tempDF, tempDF,how='outer',on='indi')
    tempTrans = tempTrans[(tempTrans['chara_x'] != tempTrans['chara_y']) | (tempTrans['lpos_x'] != tempTrans['lpos_y'])  | (tempTrans['rpos_x'] != tempTrans['rpos_y']) ]
    
    try:
        transM = transM.append(tempTrans)
    except:
        transM = pandas.DataFrame(tempTrans)
        
    try:
        rw3 = rw3.append(tempDF)
    except NameError:
        rw3 = pandas.DataFrame(tempDF)
        print(rw3)
        
rw3 = rw3.reset_index(drop=True)
rw3
transM

,source,target,pos,card
1,tavaramerkki,tavaramerkeiksi,N,PL
7,meriantura,merianturoiksi,N,PL
8,kurssilainen,kurssilaisella,N,SG
9,jokivarsi,jokivarsilta,N,PL
11,merihirviö,merihirviöitä,N,PL
12,vihersipuli,vihersipuleiksi,N,PL
13,tarkkailu,tarkkailuna,N,SG
14,staalo,staaloiksi,N,PL
19,kausatiivi,kausatiivien,N,PL
20,koputus,koputus,N,SG


In [8]:
def lcs(SB,TB):
    S = SB
    T = TB
    m = len(S)
    n = len(T)
    
    
    globList = list()
    
    while True:
        lcs_set = set()
        lcsList = list()
        counter = [[0]*(n+1) for x in range(m+1)]
        longest = 0
        for i in range(m):
            for j in range(n):
                if S[i] == T[j]:
                    c = counter[i][j] + 1
                    counter[i+1][j+1] = c
                    if c > longest:
                        lcs_set = set()
                        longest = c
                        lcs_set.add(S[i-c+1:i+1])
                    elif c == longest:
                        lcs_set.add(S[i-c+1:i+1])
        
        lcsList = list(lcs_set)    
        globList.extend(lcsList)
        if len(lcs_set) == 0:
            break
        for item in lcsList:
            S = S.replace(item,'_',1)
            T = T.replace(item,'$',1)
            m = len(S)
            n = len(T)
    
    return [item for item in globList if len(item) > 1]    

In [31]:
rw4= pandas.DataFrame()
for i,item in pd.iterrows():
    charPairs = list()
#     maxSeq = lcs(item['source'],item['target'])
    maxSeq = lcs('rataipingt', 'rntaiiingng')
#     print(maxSeq)

#     str1,str2 = item['source'],item['target']
    str1,str2 = 'rataipingt', 'rntaiiingng'

    for pat in maxSeq:
        str1 = str1.replace(pat,'$'*len(pat),1)
        str2 = str2.replace(pat,'_'*len(pat),1)
    print(str1, str2)
    str1Rems = [stuff for stuff in str1.split('$') if len(stuff) > 0]
    str2Rems = [stuff for stuff in str2.split('_') if len(stuff) > 0]
    for  indx,chara in enumerate(str1):
        if chara != '$':
            if (len(str1)-indx)*-1 < 0:
                charPairs.append([i,'del',chara,indx+1,(len(str1)-indx)*-1])
            else:
                charPairs.append([i,'del',chara,indx+1,((len(str1)-indx)*-1)+1])
    for  indx,chara in enumerate(str2):
        if chara != '_':
            if (len(str1)-indx)*-1 < 0:
                charPairs.append([i,'del',chara,indx+1,(len(str1)-indx)*-1])
            else:
                charPairs.append([i,'del',chara,indx+1,((len(str1)-indx)*-1)+1])
    tempDF = pandas.DataFrame.from_records(charPairs,columns=['indi','opn','chara','lpos','rpos'])

    try:
        rw4 = rw4.append(tempDF)
    except NameError:
        rw4 = pandas.DataFrame(tempDF)
        print(rw4)

rw4 = rw4.reset_index(drop=True)
rw4

['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing']
ra$$$p$$$t rn___i___ng
['tai', 'ing

,indi,opn,chara,lpos,rpos
0,1,del,r,1,-10
1,1,del,a,2,-9
2,1,del,p,6,-5
3,1,del,t,10,-1
4,1,del,r,1,-10
5,1,del,n,2,-9
6,1,del,i,6,-5
7,1,del,n,10,-1
8,1,del,g,11,1
9,7,del,r,1,-10
